In [2]:
import random
from IPython.display import clear_output



In [21]:
suits = ['Spades','Hearts','Diamonds','Clubs']
ranks = ['Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King','Ace']
values = {'Two':2,'Three':3,'Four':4,'Five':5,'Six':6,'Seven':7,'Eight':8,'Nine':9,'Ten':10,'Jack':10,
          'Queen':10,'King':10,'Ace':11}

class Card():
    
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.value = values[rank]
    
    def __str__(self):
        return "%s of %s" % (self.rank, self.suit)

In [28]:
class Deck():
    
    def __init__(self):
        self.cards = []
        for rank in ranks:
            for suit in suits:
                self.cards.append(Card(rank,suit))
        
    def shuffle_deck(self):
        random.shuffle(self.cards)
        
    def draw_card(self):
        return self.cards.pop(0)
    
    def add_cards(self,new_cards):
        if type(new_cards) == list:
            self.cards.extend(new_cards)
        else:
            self.cards.append(new_cards)
        
    def __str__(self):
        deck_str = ''
        for card in self.cards:
            card_str = '%s of %s' % (card.rank,card.suit)
            deck_str = deck_str + card_str + '\n'
        return deck_str


In [29]:
test_deck = Deck()


In [165]:
class Player():
    
    def __init__(self,name,balance):
        self.name = name
        self.balance = balance
        
    def place_bet(self):
        ask_for_input = True
        while ask_for_input:
            try:
                bet = int(input(f"How much do you want to bet? You have {self.balance} available."))
            except:
                print("Please enter a number ")
            else:
                if bet > self.balance:
                    print(f"Insufficient funds! You have {self.balance} available.")
                elif bet <= 0:
                    print("Specify a positive number ")
                else:
                    ask_for_input = False
        return bet
    
    def add_to_balance(self,amount):
        self.balance += amount

    def remove_from_balance(self,amount):
        self.balance -= amount

In [18]:
test_player = Player("Carl",500)
test_player.place_bet()
test_player.add_to_balance(500)
test_player.place_bet()
test_player.remove_from_balance(300)
test_player.place_bet()

How much do you want to bet? 600
Insufficient funds! 
How much do you want to bet? 300
How much do you want to bet? 801
How much do you want to bet? 701
Insufficient funds! 
How much do you want to bet? 700


700

In [177]:
class Board():
    
    def __init__(self,player):
        self.deck = Deck()
        self.house = []
        self.field = []
        self.player = player
        self.bet = 0
        
    def start_round(self):
        self.house.append(self.deck.draw_card())
        self.house.append(self.deck.draw_card())
        self.field.append(self.deck.draw_card())
        self.field.append(self.deck.draw_card())
        
    def ask_for_hit(self):
        continue_asking = True
        while continue_asking:
            player_choice = input("Receive another card (Y/N)? ")
            player_choice = player_choice.upper()
            if player_choice not in ['Y','N']:
                print("Please select 'Y' or 'N'.") 
            elif player_choice == 'Y':
                return True
            else:
                return False
    
    def hit_actor(self,actor):
        if actor == self.player.name:
            self.field.append(self.deck.draw_card())
        else:
            self.house.append(self.deck.draw_card())
        
    def give_scores(self,actor):
        if actor == self.player.name:
            cards = self.field
        elif actor == "House":
            cards = self.house
        total_score = sum(list(map(lambda card: card.value,cards)))
        if total_score <= 21:
            return total_score
        elif total_score > 21:
            num_aces = len(list(filter(lambda card: card.rank == 'Ace',cards)))
            excess_score = total_score - 21
            excess_factor = ((excess_score - 1) - (excess_score - 1) % 10)/10 + 1
            total_score = total_score - min(num_aces,excess_factor)*10
            return total_score

    def display_cards(self,actor):
        if actor == self.player.name:
            cards = self.field
        elif actor == "House":
            cards = self.house
        cards_str = ''
        for card in cards:
            card_str = '%s of %s' % (card.rank,card.suit)
            cards_str = cards_str + card_str + '\n'
        print(cards_str)
        
    def display_board(self,visibility):
        
        if visibility == "Hidden":
            cards_str = 'The house has the following cards on the board:\nThis card is not visible.\n'
            cards = self.house
            for card in cards[1:]:
                card_str = '%s of %s' % (card.rank,card.suit)
                cards_str = cards_str + card_str + '\n'
            print(cards_str)
            cards_str = self.player.name+' has the following cards on the board:\n'
            cards = self.field
            for card in cards:
                card_str = '%s of %s' % (card.rank,card.suit)
                cards_str = cards_str + card_str + '\n'
            cards_str = cards_str + "%s's score is %d\n" % (self.player.name, self.give_scores(self.player.name))
            print(cards_str)
        elif visibility == "Visible":
            cards_str = 'The house has the following cards on the board:\n'
            cards = self.house
            for card in cards:
                card_str = '%s of %s' % (card.rank,card.suit)
                cards_str = cards_str + card_str + '\n'
            cards_str = cards_str + "The House's score is %d\n" % self.give_scores("House")
            print(cards_str)
            cards_str = self.player.name+' has the following cards on the board:\n'
            cards = self.field
            for card in cards:
                card_str = '%s of %s' % (card.rank,card.suit)
                cards_str = cards_str + card_str + '\n'
            cards_str = cards_str + "%s's score is %d\n" % (self.player.name, self.give_scores(self.player.name))
            print(cards_str)
        
    def take_bet(self):
        bet = self.player.place_bet()
        self.player.remove_from_balance(bet)
        self.bet = bet
    
    def clear_board(self):
        self.deck.add_cards(self.field)
        self.field = []
        self.deck.add_cards(self.house)
        self.house = []
    
    def clear_bet(self,winner):
        if winner == self.player.name:
            self.player.add_to_balance(2*self.bet)
            print(f"{self.player.name} has won the round, adding {self.bet} to his/her balance!")
        elif winner == "None":
            self.player.add_to_balance(self.bet)
            print("No-one has won the round, resetting the bets")
        else:
            print(f"{self.player.name} has lost the round, removing {self.bet} from his/her balance!")
            pass
        self.bet = 0
        
    def ask_game_on(self):
        if self.player.balance > 0:
            continue_asking = True
        else:
            print("You are broke!")
            return False
        while continue_asking:
            player_choice = input("Do you want to play a game of BlackJack (Y/N)? ")
            player_choice = player_choice.upper()
            if player_choice not in ['Y','N']:
                print("Please select 'Y' or 'N'.") 
            elif player_choice == 'Y':
                return True
            else:
                return False
    
    def check_winner(self):
        player_score = self.give_scores(self.player.name)
        house_score = self.give_scores("House")
        if player_score > 21:
            return "House"
        elif house_score > 21:
            return self.player.name
        elif house_score > player_score:
            return "House"
        elif player_score > house_score:
            return self.player.name
        else:
            return "None"

In [124]:
test_player = Player("Carl",500)
test_board = Board(test_player)
print(len(test_board.deck.cards))
test_board.start_round()
print(len(test_board.deck.cards))
print(test_board.house[0])
print(test_board.house[1])
test_board.house[0] = Card("Ace","Spades")
test_board.house[1] = Card("Ace","Spades")
test_board.house.append(Card("Ten","Spades"))
test_board.give_scores("House")
test_board.display_cards("House")
test_board.display_board()

52
48
Nine of Spades
Seven of Spades
Ace of Spades
Ace of Spades
Ten of Spades

The house has the following cards on the board:
This card is not visible.
Ace of Spades
Ten of Spades

Carl has the following cards on the board:
Nine of Diamonds
Six of Spades
Carl's score is 15


In [194]:


# Game logic:


# Initialize the player, his name, and how much money he has
player = Player("Carl",100000)
bj_board = Board(player)
# Ask if the player wants to play
# Start running
running = bj_board.ask_game_on()
while running:
    bj_board.deck.shuffle_deck()
    bj_board.take_bet()
    bj_board.start_round()
    player_draws = True
    while player_draws:
        visibility = "Hidden"
        clear_output()
        bj_board.display_board(visibility)
        player_draws = bj_board.ask_for_hit()
        if player_draws:
            bj_board.hit_actor(bj_board.player.name)
            player_draws = bj_board.give_scores(bj_board.player.name)<21
    house_draws = bj_board.give_scores(bj_board.player.name)<=21 and bj_board.give_scores("House") < bj_board.give_scores(bj_board.player.name) and bj_board.give_scores("House") < 17 
    visibility = "Visible"
    while house_draws:
        bj_board.hit_actor("House")
        house_draws = bj_board.give_scores("House") < bj_board.give_scores(bj_board.player.name) and bj_board.give_scores("House") < 17
    clear_output()
    bj_board.display_board(visibility)
    winner = bj_board.check_winner()
    bj_board.clear_board()
    bj_board.clear_bet(winner)
    running = bj_board.ask_game_on()
# Initialize the board - deck is shuffled an house and player fields are empty
# Take player bet, check first to see if he is broke
# Draw first cards
# While player draws:
    # Check break: at/above 21
    # Ask for hit
    # Break on no hit
# While house draws:
    # 

The house has the following cards on the board:
Ace of Clubs
Seven of Hearts
The House's score is 18

Carl has the following cards on the board:
Four of Hearts
Ten of Clubs
King of Diamonds
Carl's score is 24

Carl has lost the round, removing 5000 from his/her balance!
Do you want to play a game of BlackJack (Y/N)? n


In [192]:
u = 3
while u:
    print(u)
    u-=1

3
2
1
